# Handling Item Repetitions in Study Lists

Demonstrate utilities to map study positions, remove repeated recalls, and build shuffled control datasets.

These helpers support analyses when an item can appear multiple times in a study list.
They include functions to enumerate all valid study positions for a recalled item,
remap recalls to the item's first position (for control analyses), drop repeated recalls within a trial,
and construct a shuffled control dataset aligned to mixed-list presentations.

In [1]:
from jax import numpy as jnp

from jaxcmr.repetition import (
    item_to_study_positions,
    all_study_positions,
    filter_repeated_recalls,
    relabel_trial_to_firstpos,
    make_control_dataset,
)


In [2]:
# Parameters
presentation = jnp.array([1, 2, 1, 3, 4, 5], dtype=jnp.int32)
size = 3
rec_row = jnp.array([3, 1, 0, 4, 0, 0], dtype=jnp.int32)

L = 4
toy = {
    'subject': jnp.array([[1],[1],[2],[2]], dtype=jnp.int32),
    'listLength': jnp.array([[L],[L],[L],[L]], dtype=jnp.int32),
    'pres_itemnos': jnp.array([[1,2,3,4],[1,2,1,4],[1,2,3,4],[2,1,2,3]], dtype=jnp.int32),
    'recalls': jnp.array([[1,2,3,0],[3,1,1,0],[1,1,2,0],[2,3,0,0]], dtype=jnp.int32),
    'list_type': jnp.array([[0],[1],[0],[1]], dtype=jnp.int32),
}


In [3]:
# Demo: study position mapping
item_to_study_positions(1, presentation, size)


Array([1, 3, 0], dtype=int32)

In [4]:
# Demo: all positions from a study position
all_study_positions(3, presentation, size)


Array([1, 3, 0], dtype=int32)

In [5]:
# Demo: relabel and filter repeated recalls
relabel_trial_to_firstpos(rec_row, presentation)


Array([1, 1, 0, 4, 0, 0], dtype=int32)

In [6]:
filter_repeated_recalls(jnp.vstack([rec_row, rec_row]))


Array([[3, 1, 0, 4, 0, 0],
       [3, 1, 0, 4, 0, 0]], dtype=int32)

In [7]:
# Demo: build a control dataset
ctrl = make_control_dataset(
    data=toy,
    mixed_query="data['list_type'].flatten() == 1",
    control_query="data['list_type'].flatten() == 0",
    n_shuffles=1,
    seed=0,
)
{k: v.shape for k, v in ctrl.items()}


{'subject': (2, 1),
 'recalls': (2, 4),
 'pres_itemnos': (2, 4),
 'listLength': (2, 1),
 'list_type': (2, 1)}